# Using WordNet in our Experiment

In [42]:
### this code is messy and has inconsistensies. i lost a lot of variables in the process of cleaning up the code, and so some parts may be repetitive/redundant, thanks to chatgpt troubleshooting. it was hard to retrieve the version that i originally worked on

In [43]:
import nltk
import random
assert(nltk.download('wordnet'))
nltk.download('omw-1.4')
from nltk.corpus import wordnet as wn
from collections import Counter
from datasets import load_dataset
import pprint

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [44]:
dataset = load_dataset("wikitext", "wikitext-103-v1")
print(dataset)

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 1801350
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})


In [45]:
dataset = load_dataset("wikitext", "wikitext-103-v1")
with open("wikitext_train.txt", "w", encoding="utf-8") as f:
    for line in dataset["train"]["text"]:
        if line.strip():
            f.write(line + "\n")


In [46]:
from datasets import load_dataset

dataset = load_dataset("wikitext", "wikitext-103-v1")
texts = dataset["train"]["text"]

In [47]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

# Separating by Frequency

In [48]:
freq_counter = Counter()

for line in texts:
    tokens = line.lower().split()
    freq_counter.update(tokens)

valid_words = []
for word in freq_counter:
    if wn.synsets(word):
        valid_words.append(word)

valid_words_sorted = sorted(valid_words, key=lambda w: freq_counter[w], reverse=True)

num_words = len(valid_words_sorted)
top_n = int(num_words * 0.10)
bottom_n = int(num_words * 0.10)

top_10_percent = valid_words_sorted[:top_n]
bottom_10_percent = valid_words_sorted[-bottom_n:]

print(top_10_percent[:20])
print(bottom_10_percent[:20])


['in', 'a', 'was', 'on', 'as', 'by', 'is', 'at', 'he', 'it', 'were', 'an', 'had', 'be', 'but', 'are', 'first', 'not', 'after', 'one']
['thalweg', 'grovelling', 'louisianans', 'muckraker', 'duckboards', 'hangouts', 'caruncle', 'fivesome', 'gyration', 'serranus', 'stayers', 'makins', 'dairyman', 'trundled', 'hypnotherapy', 'sponged', 'scraggly', 'ostracise', 'spokeswomen', 'perplexities']


## Separating by Frequency and Polysemy
This creates a large list that we manually went through to select our target words.

In [49]:
def get_wordnet_pos(word):
    """Return a set of POS tags for a word in WordNet."""
    pos_set = set()
    for syn in wn.synsets(word):
        pos_set.add(syn.pos())
    return ','.join(sorted(pos_set))
top_word_info = []
for word in top_10_percent:
    num_synsets = len(wn.synsets(word))
    frequency = freq_counter[word]
    pos_tags = get_wordnet_pos(word)
    top_word_info.append((word, num_synsets, frequency, pos_tags))

bottom_word_info = []
for word in bottom_10_percent:
    num_synsets = len(wn.synsets(word))
    frequency = freq_counter[word]
    pos_tags = get_wordnet_pos(word)
    bottom_word_info.append((word, num_synsets, frequency, pos_tags))

top_sorted_by_synsets = sorted(top_word_info, key=lambda x: x[1], reverse=True)
bottom_sorted_by_synsets = sorted(bottom_word_info, key=lambda x: x[1], reverse=True)

print("Top 10% words sorted by number of WordNet synsets (first 20 shown):")
for word, syn_count, freq, pos in top_sorted_by_synsets[:20]:
    print(f"{word:15}  synsets: {syn_count:<3}  frequency: {freq:<5}  POS: {pos}")

print("Bottom 10% words sorted by number of WordNet synsets (first 20 shown):")
for word, syn_count, freq, pos in bottom_sorted_by_synsets[:20]:
    print(f"{word:15}  synsets: {syn_count:<3}  frequency: {freq:<5}  POS: {pos}")


Top 10% words sorted by number of WordNet synsets (first 20 shown):
break            synsets: 75   frequency: 7769   POS: n,v
breaks           synsets: 75   frequency: 1866   POS: n,v
broken           synsets: 72   frequency: 6022   POS: a,s,v
cut              synsets: 70   frequency: 11105  POS: a,n,s,v
cuts             synsets: 61   frequency: 1946   POS: n,v
broke            synsets: 60   frequency: 7035   POS: s,v
breaking         synsets: 60   frequency: 3981   POS: n,v
run              synsets: 57   frequency: 22597  POS: n,v
runs             synsets: 57   frequency: 15050  POS: n,v
cutting          synsets: 54   frequency: 2261   POS: n,s,v
made             synsets: 52   frequency: 78666  POS: a,s,v
play             synsets: 52   frequency: 29615  POS: n,v
making           synsets: 52   frequency: 22462  POS: n,v
running          synsets: 52   frequency: 10999  POS: a,n,s,v
plays            synsets: 52   frequency: 7018   POS: n,v
make             synsets: 51   frequency: 31909 

# Frequent and Polysemous Words

In [50]:
wikitext_path = "wikitext_train.txt"
texts = []

with open(wikitext_path, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if line:
            texts.append(line.lower())
freq_counter = Counter()
for line in texts:
    tokens = line.split()
    freq_counter.update(tokens)

wn_lemmas = set(wn.all_lemma_names())

def is_valid_word(token):
    if not token:
        return False
    if token.startswith("##"):
        return False
    if not token.isalpha():
        return False
    if token != token.lower():
        return False
    if len(token) < 3:
        return False
    if token not in wn_lemmas:
        return False
    return True

monosemous_words = []

for word, freq in freq_counter.most_common():
    if is_valid_word(word):
        synsets = wn.synsets(word)
        if len(synsets) == 1:
            syn = synsets[0]
            pos = syn.pos()
            pos_map = {'n': 'noun', 'v': 'verb', 'a': 'adjective', 's': 'adjective satellite', 'r': 'adverb'}
            readable_pos = pos_map.get(pos, pos)
            monosemous_words.append((word, freq, readable_pos))

print(f"Found {len(monosemous_words)} monosemous valid words in Wikitext.\n")
print("Most frequent monosemous words (top 100 shown):")
for word, freq, pos in monosemous_words[:100]:
    print(f"{word:15}  frequency: {freq:<8}  POS: {pos}")


Found 24706 monosemous valid words in Wikitext.

Most frequent monosemous words (top 100 shown):
but              frequency: 211715    POS: adverb
not              frequency: 198856    POS: adverb
also             frequency: 184893    POS: adverb
who              frequency: 159712    POS: noun
while            frequency: 107147    POS: noun
both             frequency: 70689     POS: adjective satellite
many             frequency: 66193     POS: adjective
september        frequency: 45264     POS: noun
another          frequency: 44200     POS: adjective satellite
october          frequency: 41416     POS: noun
june             frequency: 39179     POS: noun
july             frequency: 38497     POS: noun
november         frequency: 38234     POS: noun
april            frequency: 37482     POS: noun
york             frequency: 36284     POS: noun
december         frequency: 35318     POS: noun
january          frequency: 34772     POS: noun
within           frequency: 33321     POS: adv

## Frequent and Polysemous words list


### Final Frequent and Polysemous words list
These words were manually selected to account for POS to the best of our ability

1. break
2. cut
3. running
4. play
5. make
6. better
7. light
8. falls
9. clear
10. open


## Confirming their # of similar words


In [51]:
final_polysemy = ['break', 'cut', 'running', 'play', 'make', 'better', 'light', 'falls', 'clear', 'open']
vocab = set()
for line in texts:
    tokens = line.lower().split()
    for token in tokens:
        vocab.add(token)

vocab = list(vocab)

def get_similar_words(word, num_words=10):
    related = set()
    synsets = wn.synsets(word)
    for syn in synsets:
        for lemma in syn.lemmas():
            if lemma.name().lower() != word:
                related.add(lemma.name().lower())
        for hyper in syn.hypernyms() + syn.hyponyms():
            for lemma in hyper.lemmas():
                related.add(lemma.name().lower())
    related = list(related)
    random.shuffle(related)
    return related[:num_words]

def get_dissimilar_words(word, num_words=10):
    choices = set()
    while len(choices) < num_words:
        candidate = random.choice(vocab)
        if candidate != word:
            choices.add(candidate)
    return list(choices)

word_dict = {}

for word in final_polysemy:
    word_dict[word] = {}
    word_dict[word]['similar_words'] = get_similar_words(word, 10)
    word_dict[word]['dissimilar_words'] = get_dissimilar_words(word, 10)

for w in final_polysemy:
    print(f"\n{w}:")
    print("  similar_words:", word_dict[w]['similar_words'])
    print("  dissimilar_words:", word_dict[w]['dissimilar_words'])



break:
  similar_words: ['pause', 'interval', 'give_the_bounce', 'disclose', 'shot', 'trip_the_light_fantastic', 'break_away', 'break_out', 'misfunction', 'falling_out']
  dissimilar_words: ['denisof', 'endzone', 'hayter', 'fame', 'rubbing', 'eishima', 'smashers', 'præn', 'artepiazza', 'romema']

cut:
  similar_words: ['ruffle', 'knap', 'curve', 'weaken', 'plane', 'grow', 'dissection', 'abridge', 'flip', 'cut_down']
  dissimilar_words: ['elantra', 'trireme', 'piggyback', 'ev', 'councilwoman', 'corneal', 'cores', 'velshi', 'weepingwillow17', 'kingsglaive']

running:
  similar_words: ['football_play', 'rabbit', 'propagate', 'carry_through', 'fulfill', 'treat', 'swirl', 'filter', 'spread_out', 'go_deep']
  dissimilar_words: ['383b', 'quinquennial', 'etherians', 'itching', 'resemblant', 'valise', 'vangshylla', 'onecommunity', 'firebenders', 'bookwalter']

play:
  similar_words: ['backstop', 'slackness', 'flirtation', 'symphonise', 'walk', 'hole', 'wit', 'morality_play', 'recapitulate', 'u

## Confirm if in Bert Vocab:

In [52]:
bert_vocab_path = "vocab.txt"

bert_vocab = set()
with open(bert_vocab_path, "r", encoding="utf-8") as f:
    for line in f:
        token = line.strip()
        if token:
            bert_vocab.add(token)

In [53]:
for word in poly_freq:
  if word in bert_vocab:
    print(f"{word} is in the BERT vocab.")

break is in the BERT vocab.
cut is in the BERT vocab.
running is in the BERT vocab.
play is in the BERT vocab.
make is in the BERT vocab.
better is in the BERT vocab.
light is in the BERT vocab.
falls is in the BERT vocab.
clear is in the BERT vocab.
open is in the BERT vocab.


# Similar Word Finder
Used to manually check each word's similar word number before determining if it is added to the list or not.

In [54]:
vocab_file = "vocab.txt"
bert_vocab = set()

with open(vocab_file, "r", encoding="utf-8") as f:
    for line in f:
        token = line.strip()
        if token.isalpha() and token.islower():
            bert_vocab.add(token)

print(f"Loaded {len(bert_vocab)} BERT vocab words.")

def wordnet_similar_in_bert(word):
    word_lower = word.lower()
    related = set()

    for syn in wn.synsets(word_lower):
        for lemma in syn.lemmas():
            related.add(lemma.name().lower())
        for hyper in syn.hypernyms():
            for lemma in hyper.lemmas():
                related.add(lemma.name().lower())
        for hypo in syn.hyponyms():
            for lemma in hypo.lemmas():
                related.add(lemma.name().lower())

    related.discard(word_lower)
    filtered = [w for w in related if w in bert_vocab]

    return filtered

input_word = input("Enter a word: ").strip().lower()
similar_words = wordnet_similar_in_bert(input_word)

print(f"Number of WordNet-based similar words in BERT vocab for '{input_word}': {len(similar_words)}")
print("Similar words:", similar_words)


Loaded 11338 BERT vocab words.
Enter a word: open
Number of WordNet-based similar words in BERT vocab for 'open': 34
Similar words: ['gap', 'outside', 'found', 'capable', 'launch', 'yield', 'jimmy', 'opened', 'subject', 'start', 'breach', 'country', 'lance', 'go', 'candid', 'prize', 'loose', 'clear', 'turn', 'surface', 'area', 'establish', 'grass', 'exposed', 'exhibit', 'spread', 'butterfly', 'move', 'tournament', 'afford', 'give', 'overt', 'exterior', 'display']


# Infrequent and Polysemous words

## Infrequent and Polysemous words list

In [55]:
wikitext_path = "wikitext_train.txt"

with open(wikitext_path, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if line:
            texts.append(line.lower())
freq_counter = Counter()
for line in texts:
    tokens = line.split()
    freq_counter.update(tokens)

wn_lemmas = set(wn.all_lemma_names())

def is_valid_word(token):
    if not token:
        return False
    if token.startswith("##"):
        return False
    if not token.isalpha():
        return False
    if token != token.lower():
        return False
    if len(token) < 3:
        return False
    if token not in wn_lemmas:
        return False
    return True

monosemous_words = []
polysemous_words = []

for word, freq in freq_counter.most_common():
    if is_valid_word(word):
        synsets = wn.synsets(word)
        if len(synsets) == 1:
            syn = synsets[0]
            pos = syn.pos()
            pos_map = {'n': 'noun', 'v': 'verb', 'a': 'adjective', 's': 'adjective satellite', 'r': 'adverb'}
            readable_pos = pos_map.get(pos, pos)
            monosemous_words.append((word, freq, readable_pos))
        elif len(synsets) > 1:
            polysemous_words.append((word, freq, len(synsets)))

print(f"Found {len(monosemous_words)} monosemous and {len(polysemous_words)} polysemous valid words in Wikitext.\n")

print("Top 100 most frequent monosemous words:")
for word, freq, pos in monosemous_words[:100]:
    print(f"{word:15}  frequency: {freq:<8}  POS: {pos}")

print("\nTop 100 polysemous words (by frequency):")
for word, freq, sense_count in polysemous_words[:100]:
    print(f"{word:15}  frequency: {freq:<8}  senses: {sense_count}")

infrequent_words = [w for w in polysemous_words if w[1] <= 2]
print(f"\nFound {len(infrequent_words)} infrequent (freq ≤ 2) polysemous words.")
for word, freq, sense_count in infrequent_words[:50]:
    print(f"{word:15}  frequency: {freq:<8}  senses: {sense_count}")


Found 24706 monosemous and 22997 polysemous valid words in Wikitext.

Top 100 most frequent monosemous words:
but              frequency: 423430    POS: adverb
not              frequency: 397712    POS: adverb
also             frequency: 369786    POS: adverb
who              frequency: 319424    POS: noun
while            frequency: 214294    POS: noun
both             frequency: 141378    POS: adjective satellite
many             frequency: 132386    POS: adjective
september        frequency: 90528     POS: noun
another          frequency: 88400     POS: adjective satellite
october          frequency: 82832     POS: noun
june             frequency: 78358     POS: noun
july             frequency: 76994     POS: noun
november         frequency: 76468     POS: noun
april            frequency: 74964     POS: noun
york             frequency: 72568     POS: noun
december         frequency: 70636     POS: noun
january          frequency: 69544     POS: noun
within           frequency: 66642

After using the Similar Word Finder, we searched for words that did fit the criteria.

In [56]:
vocab_file = "vocab.txt"
bert_vocab = set()

with open(vocab_file, "r", encoding="utf-8") as f:
    for line in f:
        token = line.strip()
        if token.isalpha() and token.islower():
            bert_vocab.add(token)

freq_counter = Counter()
for line in texts:
    tokens = line.split()
    for token in tokens:
        token_lower = token.lower()
        if token_lower.isalpha() and token_lower in bert_vocab:
            freq_counter[token_lower] += 1

def get_wordnet_similar_in_bert(word):
    """Return WordNet-based similar words that exist in BERT vocab."""
    related = set()
    for syn in wn.synsets(word):
        for lemma in syn.lemmas():
            related.add(lemma.name().lower())
        for hyper in syn.hypernyms():
            for lemma in hyper.lemmas():
                related.add(lemma.name().lower())
        for hypo in syn.hyponyms():
            for lemma in hypo.lemmas():
                related.add(lemma.name().lower())
    related.discard(word)
    return [w for w in related if w in bert_vocab]

def get_num_synsets(word):
    """Return number of WordNet senses."""
    return len(wn.synsets(word))

words_in_vocab = [w for w in bert_vocab if w in freq_counter]
if not words_in_vocab:
    raise ValueError("No BERT vocab words found in dataset!")

frequencies = [freq_counter[w] for w in words_in_vocab]
frequencies.sort()
threshold = frequencies[int(0.1 * len(frequencies))]
bottom_words = [w for w in words_in_vocab if freq_counter[w] <= threshold]

print(f"Found {len(bottom_words)} bottom 10% frequency words.")

candidates = []
for word in bottom_words:
    similar = get_wordnet_similar_in_bert(word)
    if len(similar) >= 10:
        candidates.append({
            "word": word,
            "num_synsets": get_num_synsets(word),
            "num_similar": len(similar),
            "similar_words": similar[:10]
        })

candidates_sorted = sorted(candidates, key=lambda x: x["num_synsets"], reverse=True)

print("\nTop polysemous, infrequent words with ≥10 similar words:\n")
for entry in candidates_sorted[:20]:
    print(f"Word: {entry['word']}")
    print(f"  # Synsets: {entry['num_synsets']}")
    print(f"  # Similar (in vocab): {entry['num_similar']}")
    print(f"  Similar words: {entry['similar_words']}\n")


Found 998 bottom 10% frequency words.

Top polysemous, infrequent words with ≥10 similar words:

Word: comed
  # Synsets: 21
  # Similar (in vocab): 45
  Similar words: ['settle', 'rise', 'be', 'pass', 'extend', 'follow', 'shore', 'become', 'number', 'aggregate']

Word: bes
  # Synsets: 14
  # Similar (in vocab): 127
  Similar words: ['promise', 'remain', 'hold', 'cost', 'lie', 'flow', 'live', 'swim', 'consist', 'buzz']

Word: hitch
  # Synsets: 12
  # Similar (in vocab): 18
  Similar words: ['ride', 'check', 'rub', 'buck', 'move', 'tie', 'tour', 'walk', 'halt', 'connection']

Word: ses
  # Synsets: 8
  # Similar (in vocab): 11
  Similar words: ['sec', 's', 'south', 'southward', 'point', 'letter', 'southeast', 'sulfur', 'element', 'se']

Word: ply
  # Synsets: 8
  # Similar (in vocab): 31
  Similar words: ['accommodate', 'employ', 'treat', 'bed', 'dish', 'fill', 'meet', 'power', 'travel', 'join']

Word: vamp
  # Synsets: 7
  # Similar (in vocab): 14
  Similar words: ['butterfly', 'manu

### Final Infrequent Polysemous words list
From the list above, the following words were chosen:

1.   ply
2.   vamp
3. cant
4. cons
4. mend
5. spiel
6. hitch
7. middles
8. crams
10. weares




## Confirm if in Bert Vocab

In [57]:
poly_infreq = ["ply", "vamp", "cant", "weares", "crams",
               "cons", "mend", "spiel", "hitch", "middles"]

In [58]:
for word in poly_infreq:
  if word in bert_vocab:
    print(f"{word} is in the BERT vocab.")

ply is in the BERT vocab.
vamp is in the BERT vocab.
cant is in the BERT vocab.
cons is in the BERT vocab.
mend is in the BERT vocab.
spiel is in the BERT vocab.
hitch is in the BERT vocab.
middles is in the BERT vocab.


# Frequent and Monosemous Words

## Frequent and Monosemous words list

In [59]:
wikitext_path = "wikitext_train.txt"
texts = []

with open(wikitext_path, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if line:
            texts.append(line.lower())

freq_counter = Counter()
for line in texts:
    tokens = line.split()
    freq_counter.update(tokens)

wn_lemmas = set(wn.all_lemma_names())

def is_valid_word(token):
    if not token:
        return False
    if token.startswith("##"):
        return False
    if not token.isalpha():
        return False
    if token != token.lower():
        return False
    if len(token) < 3:
        return False
    if token not in wn_lemmas:
        return False
    return True

monosemous_words = []
freq_threshold = 50

for word, freq in freq_counter.most_common():
    if freq < freq_threshold:
        break
    if is_valid_word(word):
        synsets = wn.synsets(word)
        if len(synsets) == 1:
            syn = synsets[0]
            pos = syn.pos()
            pos_map = {
                'n': 'noun',
                'v': 'verb',
                'a': 'adjective',
                's': 'adjective satellite',
                'r': 'adverb'
            }
            readable_pos = pos_map.get(pos, pos)
            monosemous_words.append((word, freq, readable_pos))

print(f"Found {len(monosemous_words)} frequent (freq ≥ {freq_threshold}) monosemous valid words in Wikitext.\n")
print("Most frequent monosemous words:")
for word, freq, pos in monosemous_words[:100]:
    print(f"{word:15}  frequency: {freq:<8}  POS: {pos}")


Found 8534 frequent (freq ≥ 50) monosemous valid words in Wikitext.

Most frequent monosemous words:
but              frequency: 211715    POS: adverb
not              frequency: 198856    POS: adverb
also             frequency: 184893    POS: adverb
who              frequency: 159712    POS: noun
while            frequency: 107147    POS: noun
both             frequency: 70689     POS: adjective satellite
many             frequency: 66193     POS: adjective
september        frequency: 45264     POS: noun
another          frequency: 44200     POS: adjective satellite
october          frequency: 41416     POS: noun
june             frequency: 39179     POS: noun
july             frequency: 38497     POS: noun
november         frequency: 38234     POS: noun
april            frequency: 37482     POS: noun
york             frequency: 36284     POS: noun
december         frequency: 35318     POS: noun
january          frequency: 34772     POS: noun
within           frequency: 33321     POS:

Finding additional replacement words with more similar words

In [60]:
import nltk
from nltk.corpus import wordnet as wn
from collections import Counter
from datasets import load_dataset

nltk.download('wordnet')
nltk.download('omw-1.4')

vocab_file = "vocab.txt"
bert_vocab = set()

with open(vocab_file, "r", encoding="utf-8") as f:
    for line in f:
        token = line.strip()
        if token.isalpha() and token.islower():
            bert_vocab.add(token)

print(f"Loaded {len(bert_vocab)} valid BERT vocab words.")

dataset = load_dataset("wikitext", "wikitext-103-v1")
texts = dataset["train"]["text"]

freq_counter = Counter()
for line in texts:
    tokens = line.split()
    for token in tokens:
        token_lower = token.lower()
        if token_lower.isalpha() and token_lower in bert_vocab:
            freq_counter[token_lower] += 1

def get_wordnet_similar_in_bert(word):
    """Return WordNet-based similar words that exist in BERT vocab."""
    related = set()
    for syn in wn.synsets(word):
        for lemma in syn.lemmas():
            related.add(lemma.name().lower())
        for hyper in syn.hypernyms():
            for lemma in hyper.lemmas():
                related.add(lemma.name().lower())
        for hypo in syn.hyponyms():
            for lemma in hypo.lemmas():
                related.add(lemma.name().lower())
    related.discard(word)
    return [w for w in related if w in bert_vocab]

def get_num_synsets(word):
    return len(wn.synsets(word))

words_in_vocab = [w for w in bert_vocab if w in freq_counter]
frequencies = sorted([freq_counter[w] for w in words_in_vocab])
threshold = frequencies[int(0.9 * len(frequencies))]  # top 10%
top_words = [w for w in words_in_vocab if freq_counter[w] >= threshold]

print(f"Found {len(top_words)} top 10% frequency words.")

monosemous_top_candidates = []
for word in top_words:
    num_syn = get_num_synsets(word)
    if num_syn == 1:
        similar = get_wordnet_similar_in_bert(word)
        if len(similar) >= 5:
            monosemous_top_candidates.append({
                "word": word,
                "num_synsets": num_syn,
                "frequency": freq_counter[word],
                "num_similar": len(similar),
                "similar_words": similar[:10]
            })

monosemous_top_candidates_sorted = sorted(
    monosemous_top_candidates,
    key=lambda x: x["num_similar"],
    reverse=True
)
print("\nTop 10% frequency monosemous words with ≥10 similar words:\n")
for entry in monosemous_top_candidates_sorted[:20]:
    print(f"Word: {entry['word']}")
    print(f"  Frequency: {entry['frequency']}")
    print(f"  # Synsets: {entry['num_synsets']}")
    print(f"  # Similar (in vocab): {entry['num_similar']}")
    print(f"  Similar words: {entry['similar_words']}\n")


Loaded 11338 valid BERT vocab words.


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Found 998 top 10% frequency words.

Top 10% frequency monosemous words with ≥10 similar words:

Word: victory
  Frequency: 15222
  # Synsets: 1
  # Similar (in vocab): 11
  Similar words: ['ending', 'pin', 'sweep', 'independence', 'triumph', 'conclusion', 'finish', 'win', 'success', 'fall']

Word: praised
  Frequency: 13287
  # Synsets: 1
  # Similar (in vocab): 9
  Similar words: ['laud', 'push', 'promote', 'proclaim', 'praise', 'value', 'measure', 'recommend', 'assess']

Word: highway
  Frequency: 22802
  # Synsets: 1
  # Similar (in vocab): 8
  Similar words: ['motorway', 'pike', 'expressway', 'route', 'freeway', 'road', 'bypass', 'interstate']

Word: almost
  Frequency: 16835
  # Synsets: 1
  # Similar (in vocab): 5
  Similar words: ['most', 'about', 'nearly', 'near', 'virtually']



We now add victory, praised, highway, and almost into our list for frequent monosemous words.

Final Frequent and Monosemous words list
1. tennis
2. guild
3. grossing
4. televised
5. alleged
6. victory
7. praised
8. highway
9. almost
10. mini


## Confirm if in Bert Vocab

In [61]:
mono_freq = ["tennis", "praised", "victory", "guild", "mini", "grossing",
             "highway", "televised", "alleged", "almost"]

In [62]:
for word in mono_freq:
  if word in bert_vocab:
    print(f"{word} is in the BERT vocab.")

tennis is in the BERT vocab.
praised is in the BERT vocab.
victory is in the BERT vocab.
guild is in the BERT vocab.
mini is in the BERT vocab.
grossing is in the BERT vocab.
highway is in the BERT vocab.
televised is in the BERT vocab.
alleged is in the BERT vocab.
almost is in the BERT vocab.


# Infrequent Monosemous Words
These words were randomly selected based on their POS to control for it. ** Edit to add: there is an error in this code, as the words are selected without considering if they are in our XT LTG Bert's vocab.

In [63]:
from nltk.corpus import wordnet as wn

bottom_10_percent_nouns_info = []

for word in bottom_10_percent:
    if not word.islower():
        continue

    synsets = wn.synsets(word)

    noun_synsets = [syn for syn in synsets if syn.pos() == 'n' or syn.pos() == 'v']

    if len(noun_synsets) > 0:
        num_senses = len(noun_synsets)
        pos_tags = [syn.pos() for syn in noun_synsets]

        bottom_10_percent_nouns_info.append({
            'word': word,
            'num_synsets': num_senses,
            'pos_tags': pos_tags
        })

for info in bottom_10_percent_nouns_info[:50]:
    print(f"Word: {info['word']}")
    print(f"  Number of noun synsets: {info['num_synsets']}")
    print(f"  POS tags: {info['pos_tags']}")
    print("-" * 40)


Word: thalweg
  Number of noun synsets: 2
  POS tags: ['n', 'n']
----------------------------------------
Word: grovelling
  Number of noun synsets: 1
  POS tags: ['v']
----------------------------------------
Word: louisianans
  Number of noun synsets: 1
  POS tags: ['n']
----------------------------------------
Word: muckraker
  Number of noun synsets: 1
  POS tags: ['n']
----------------------------------------
Word: duckboards
  Number of noun synsets: 1
  POS tags: ['n']
----------------------------------------
Word: hangouts
  Number of noun synsets: 1
  POS tags: ['n']
----------------------------------------
Word: caruncle
  Number of noun synsets: 1
  POS tags: ['n']
----------------------------------------
Word: fivesome
  Number of noun synsets: 2
  POS tags: ['n', 'n']
----------------------------------------
Word: gyration
  Number of noun synsets: 2
  POS tags: ['n', 'n']
----------------------------------------
Word: serranus
  Number of noun synsets: 1
  POS tags: ['n']

In [64]:
import random
from nltk.corpus import wordnet as wn

eligible_monosem_nouns = []
eligible_monosem_verbs = []

for info in bottom_10_percent_nouns_info:
    if info['num_synsets'] == 1:
        synset_pos = info['pos_tags'][0]
        if synset_pos == 'n':
            eligible_monosem_nouns.append(info['word'])
        elif synset_pos == 'v':
            eligible_monosem_verbs.append(info['word'])

infreq_monosem_nouns = random.sample(eligible_monosem_nouns, min(5, len(eligible_monosem_nouns)))
infreq_monosem_verbs = random.sample(eligible_monosem_verbs, min(5, len(eligible_monosem_verbs)))

print("Randomly selected infrequent monosemous nouns:", infreq_monosem_nouns)
print("Randomly selected infrequent monosemous verbs:", infreq_monosem_verbs)


Randomly selected infrequent monosemous nouns: ['masoud', 'pasto', 'sulfonate', 'calendula', 'prole']
Randomly selected infrequent monosemous verbs: ['undersigned', 'idolise', 'gnashed', 'aestivating', 'construing']


Some words did not have enough pre-existing similar words, so they were replaced by other ones that had more.

In [65]:
vocab_file = "vocab.txt"
bert_vocab = set()

with open(vocab_file, "r", encoding="utf-8") as f:
    for line in f:
        token = line.strip()
        if token.isalpha() and token.islower():
            bert_vocab.add(token)

print(f"{len(bert_vocab)} valid BERT vocab words.")

freq_counter = Counter()
for line in texts:
    tokens = line.split()
    for token in tokens:
        token_lower = token.lower()
        if token_lower.isalpha() and token_lower in bert_vocab:
            freq_counter[token_lower] += 1

def get_wordnet_similar_in_bert(word):
    """Return WordNet-based similar words that exist in BERT vocab."""
    related = set()
    for syn in wn.synsets(word):
        for lemma in syn.lemmas():
            related.add(lemma.name().lower())
        for hyper in syn.hypernyms():
            for lemma in hyper.lemmas():
                related.add(lemma.name().lower())
        for hypo in syn.hyponyms():
            for lemma in hypo.lemmas():
                related.add(lemma.name().lower())
    related.discard(word)
    return [w for w in related if w in bert_vocab]

def get_num_synsets(word):
    return len(wn.synsets(word))

words_in_vocab = [w for w in bert_vocab if w in freq_counter]
frequencies = [freq_counter[w] for w in words_in_vocab]
frequencies.sort()
threshold = frequencies[int(0.1 * len(frequencies))]
bottom_words = [w for w in words_in_vocab if freq_counter[w] <= threshold]

print(f"{len(bottom_words)} bottom 10% frequency words.")

monosemous_candidates = []
for word in bottom_words:
    num_syn = get_num_synsets(word)
    if num_syn == 1:
        similar = get_wordnet_similar_in_bert(word)
        if len(similar) >= 3:
            monosemous_candidates.append({
                "word": word,
                "num_synsets": num_syn,
                "num_similar": len(similar),
                "similar_words": similar[:10]
            })

monosemous_candidates_sorted = sorted(monosemous_candidates, key=lambda x: x["num_similar"], reverse=True)
print("\nMonosemous (1 synset) infrequent words with ≥10 similar words:\n")
for entry in monosemous_candidates_sorted[:20]:
    print(f"Word: {entry['word']}")
    print(f"  # Synsets: {entry['num_synsets']}")
    print(f"  # Similar (in vocab): {entry['num_similar']}")
    print(f"  Similar words: {entry['similar_words']}\n")


11338 valid BERT vocab words.
998 bottom 10% frequency words.

Monosemous (1 synset) infrequent words with ≥10 similar words:

Word: eff
  # Synsets: 1
  # Similar (in vocab): 11
  Similar words: ['jazz', 'bed', 'have', 'love', 'take', 'couple', 'fuck', 'know', 'bang', 'pair']

Word: bod
  # Synsets: 1
  # Similar (in vocab): 9
  Similar words: ['anatomy', 'body', 'form', 'figure', 'flesh', 'shape', 'person', 'frame', 'build']

Word: thous
  # Synsets: 1
  # Similar (in vocab): 6
  Similar words: ['g', 'k', 'yard', 'grand', 'm', 'thousand']

Word: diam
  # Synsets: 1
  # Similar (in vocab): 5
  Similar words: ['bore', 'caliber', 'diameter', 'gauge', 'length']

Word: deem
  # Synsets: 1
  # Similar (in vocab): 5
  Similar words: ['regard', 'see', 'consider', 'view', 'hold']

Word: annoy
  # Synsets: 1
  # Similar (in vocab): 5
  Similar words: ['get', 'gravel', 'rag', 'devil', 'harry']

Word: transact
  # Synsets: 1
  # Similar (in vocab): 5
  Similar words: ['trade', 'bank', 'interact'

### Final Infrequent Monosemous words list


1.   denouncements
2.   mortgagee
3. pharmacologists
4. legalises
5. behoove
6. demonise
7. transact
8. eff
9. bod
10. deem



# Confirm if in Bert Vocab:


In [66]:
final_im= ["denouncements", "mortgagee", "eff", "pharmacologists",
               "bod", "legalises", "behoove", "demonise", "transace", "deem"]

In [67]:
for word in final_im:
  if word in bert_vocab:
    print(f"{word} is in the BERT vocab.")

eff is in the BERT vocab.
bod is in the BERT vocab.
deem is in the BERT vocab.


# Putting together a master dictionary in the end

In [68]:
vocab_file = "vocab.txt"
bert_vocab = set()

with open(vocab_file, "r", encoding="utf-8") as f:
    for line in f:
        token = line.strip()
        if token.isalpha() and token.islower():
            bert_vocab.add(token)

print(f"Loaded {len(bert_vocab)} BERT vocab words.")

dataset = load_dataset("wikitext", "wikitext-103-v1")
texts = dataset["train"]["text"]

dataset_vocab = set()
for line in texts:
    tokens = line.split()
    for token in tokens:
        if token.isalpha() and token.islower():
            dataset_vocab.add(token.lower())

dataset_vocab = list(dataset_vocab)

poly_freq = ["break", "cut", "running", "play", "make", "better",
             "light", "falls", "clear", "open"]

poly_infreq = ["ply", "vamp", "cant", "weares", "crams",
               "cons", "mend", "spiel", "hitch", "middles"]

mono_freq = ["tennis", "praised", "victory", "guild", "mini", "grossing",
             "highway", "televised", "alleged", "almost"]

mono_infreq = ["denouncements", "mortgagee", "eff", "pharmacologists",
               "bod", "legalises", "behoove", "demonise", "transact", "deem"]

target_words_list = set(poly_freq + poly_infreq + mono_freq + mono_infreq)

def get_num_senses(word):
    """Return number of WordNet synsets for a word."""
    return len(wn.synsets(word))

def get_similar_words_wordnet_only(word, max_results=10):
    """Return WordNet-based similar words filtered by BERT vocab."""
    word_lower = word.lower()
    related = set()

    for syn in wn.synsets(word_lower):
        for lemma in syn.lemmas():
            related.add(lemma.name().lower())
        for hyper in syn.hypernyms():
            for lemma in hyper.lemmas():
                related.add(lemma.name().lower())
        for hypo in syn.hyponyms():
            for lemma in hypo.lemmas():
                related.add(lemma.name().lower())

    related.discard(word_lower)

    filtered = [w for w in related if w in bert_vocab]
    random.shuffle(filtered)
    return filtered[:max_results]

def ensure_ten_similar_words(similar_words, word):
    """Ensure similar_words has 10 entries; ask user to add more if needed."""
    while len(similar_words) < 10:
        print(f"\nThe word '{word}' has only {len(similar_words)} similar words:")
        print(similar_words)
        user_input = input(f"Add an additional similar word for '{word}': ").strip().lower()
        if user_input in bert_vocab and user_input not in similar_words:
            similar_words.append(user_input)
            print(f"Added '{user_input}' to similar_words.")
        else:
            print(f"'{user_input}' is not in BERT vocab or already in the list. Try again.")
    return similar_words

def get_random_words_from_bert_vocab(exclude_words, num_words=10):
    """Return random words from BERT vocab excluding target words."""
    candidates = list(bert_vocab - set(exclude_words))
    return random.sample(candidates, min(num_words, len(candidates)))

def add_words(word_list, freq_class, poly_class, target_words_set):
    entries = []
    for word in word_list:
        similar = get_similar_words_wordnet_only(word, 10)
        similar = ensure_ten_similar_words(similar, word)

        entry = {
            "word": word,
            "frequency_class": freq_class,
            "polysemy_class": poly_class,
            "num_senses": get_num_senses(word),
            "similar_words": similar,
            "random_words": get_random_words_from_bert_vocab(target_words_set.union({word.lower()}), 10)
        }
        entries.append(entry)
    return entries


mock_dataset = {"target_words": []}
mock_dataset["target_words"].extend(add_words(poly_freq, "frequent", "polysemous", target_words_list))
mock_dataset["target_words"].extend(add_words(poly_infreq, "infrequent", "polysemous", target_words_list))
mock_dataset["target_words"].extend(add_words(mono_freq, "frequent", "monosemous", target_words_list))
mock_dataset["target_words"].extend(add_words(mono_infreq, "infrequent", "monosemous", target_words_list))

pp = pprint.PrettyPrinter(indent=2, width=120)
pp.pprint(mock_dataset)


Loaded 11338 BERT vocab words.

The word 'tennis' has only 2 similar words:
['doubles', 'singles']
Add an additional similar word for 'tennis': sport
Added 'sport' to similar_words.

The word 'tennis' has only 3 similar words:
['doubles', 'singles', 'sport']
Add an additional similar word for 'tennis': duo
Added 'duo' to similar_words.

The word 'tennis' has only 4 similar words:
['doubles', 'singles', 'sport', 'duo']
Add an additional similar word for 'tennis': ball
Added 'ball' to similar_words.

The word 'tennis' has only 5 similar words:
['doubles', 'singles', 'sport', 'duo', 'ball']
Add an additional similar word for 'tennis': net
Added 'net' to similar_words.

The word 'tennis' has only 6 similar words:
['doubles', 'singles', 'sport', 'duo', 'ball', 'net']
Add an additional similar word for 'tennis': court
Added 'court' to similar_words.

The word 'tennis' has only 7 similar words:
['doubles', 'singles', 'sport', 'duo', 'ball', 'net', 'court']
Add an additional similar word for '

## Replacing the random words to use in vocab real words

In [69]:
vocab_file = "vocab.txt"
bert_vocab_raw = []

with open(vocab_file, "r", encoding="utf-8") as f:
    for line in f:
        token = line.strip()
        bert_vocab_raw.append(token)

print(f"Total tokens in vocab file: {len(bert_vocab_raw)}")

wn_lemmas = set()
for syn in wn.all_synsets():
    for lemma in syn.lemmas():
        wn_lemmas.add(lemma.name().lower())

def is_valid_word(token):
    if not token:
        return False
    if token.startswith("##"):
        return False
    if not token.isalpha():
        return False
    if token != token.lower():
        return False
    if len(token) < 3:
        return False
    if token not in wn_lemmas:
        return False
    return True

bert_vocab = {t for t in bert_vocab_raw if is_valid_word(t)}
print(f"Filtered real-word BERT vocab size: {len(bert_vocab)}")

def get_random_words_from_bert_vocab(exclude_words, num_words=10):
    """Return random full words from BERT vocab excluding target words."""
    exclude_set = set(w.lower() for w in exclude_words)
    candidates = [w for w in bert_vocab if w not in exclude_set]

    if len(candidates) < num_words:
        candidates = [w for w in bert_vocab_raw
                      if w.isalpha() and w.islower() and not w.startswith("##")
                      and len(w) >= 3 and w not in exclude_set]
    return random.sample(candidates, min(num_words, len(candidates)))

for entry in mock_dataset.get("target_words", []):
    exclude = set(x.lower() for x in target_words_list)
    exclude.add(entry["word"].lower())
    entry["random_words"] = get_random_words_from_bert_vocab(exclude, 10)

pp = pprint.PrettyPrinter(indent=2, width=100)
pp.pprint(mock_dataset)


Total tokens in vocab file: 16384
Filtered real-word BERT vocab size: 6657
{ 'target_words': [ { 'frequency_class': 'frequent',
                      'num_senses': 75,
                      'polysemy_class': 'polysemous',
                      'random_words': [ 'anne',
                                        'scar',
                                        'cold',
                                        'drought',
                                        'matthew',
                                        'stuff',
                                        'establish',
                                        'collected',
                                        'crossed',
                                        'promising'],
                      'similar_words': [ 'hurt',
                                         'divide',
                                         'dance',
                                         'work',
                                         'exchange',
                    

In [70]:
poly_freq = ["break", "cut", "running", "play", "make", "better",
             "light", "falls", "clear", "open"]

poly_infreq = ["ply", "vamp", "cant", "weares", "crams",
               "cons", "mend", "spiel", "hitch", "middles"]

mono_freq = ["tennis", "praised", "victory", "guild", "mini", "grossing",
             "highway", "televised", "alleged", "almost"]

mono_infreq = ["denouncements", "mortgagee", "eff", "pharmacologists",
               "bod", "legalises", "behoove", "demonise", "transace", "deem"]

# target_words_list = set(poly_freq + poly_infreq + mono_freq + mono_infreq)